In [ ]:
from langchain_openai import ChatOpenAI



# Define the local llama.cpp server as an OpenAI-compatible endpoint
llm = ChatOpenAI(
    openai_api_key="",
    model_name="gpt-4o-mini-2024-07-18",
    temperature=0,
    seed=42,
)

# Get the dataset

In [3]:
from datasets import load_dataset

# Load the GSM8K dataset
dataset = load_dataset("gsm8k", "main")

# Access the training and test splits
train_dataset = dataset["train"]
test_dataset = dataset["test"]

/Users/macbook/.pyenv/versions/3.11.10/envs/nlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Get a sample
val_sample = test_dataset.shuffle(seed=42).select(range(100))

# Test the model

In [5]:
def check_answer(model_answer, true_answer):
    try:
        true_answer = true_answer.split(',')
        
        true_answer = [int(x) for x in true_answer]
    except Exception:
        true_answer = [int(true_answer)]
    
    return int(model_answer) in true_answer

In [ ]:
from langchain.schema import HumanMessage, SystemMessage
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from tqdm import tqdm
import re

zero_shot = \
"""
### INSTRUCTIONS
1) Solve the following grade school level math problem step-by-step.
2) If you solve it right, I will give you a millon dollars.
3) At the end, provide the answer formatted as Answer: <ANSWER>
"""
system_message = SystemMessage(content=zero_shot)

k = 0
model_answers = []

for problem in tqdm(val_sample):
    # Get the math problem and the correct answer
    math_problem = problem['question']
    correct_answer = problem['answer'].split("### ")[1]

    human_message = HumanMessage(content=math_problem)
    
    # Generate the model's response
    model_response = llm.invoke([system_message, human_message])
    model_answers.append(model_response.content)

    # Use regex to parse the numerical answer
    try:
        model_ans = re.search(r'Answer:\s*[^0-9]*([\d]+(?:\.\d+)?)', model_response.content).group(1).strip()
        # print(model_ans)
        # print(correct_answer)
        # print('----------------')

        k += 1 if check_answer(model_ans, correct_answer) else 0
        
    except Exception:
        continue

  1%|          | 1/100 [00:03<06:15,  3.79s/it]

109
109
----------------


  2%|▏         | 2/100 [00:10<09:07,  5.59s/it]

107
89
----------------


  3%|▎         | 3/100 [00:13<07:16,  4.50s/it]

...

291
291
----------------


 99%|█████████▉| 99/100 [08:19<00:06,  6.04s/it]

70
70
----------------


100%|██████████| 100/100 [08:25<00:00,  5.05s/it]

63
63
----------------


In [7]:
print(f"Precision: {k/len(val_sample)}")

Precision: 0.85
